# Creating a Segmentation App with MONAI Deploy App SDK

This tutorial shows how to create an organ segmentation application for a PyTorch model that has been trained with MONAI.

Deploying AI models requires the integration with clinical imaging network, even if in a for-research-use setting. This means that the AI deploy application will need to support standards-based imaging protocols, and specifically for Radiological imaging, DICOM protocol.

Typically, DICOM network communication, either in DICOM TCP/IP network protocol or DICOMWeb, would be handled by DICOM devices or services, e.g. MONAI Deploy Informatics Gateway, so the deploy application itself would only need to use DICOM Part 10 files as input and save the AI result in DICOM Part10 file(s). For segmentation use cases, the DICOM instance file could be a DICOM Segmentation object or a DICOM RT Structure Set, and for classification, DICOM Structure Report and/or DICOM Encapsulated PDF.

During model training, input and label images are typically in non-DICOM volumetric image format, e.g., NIfTI and PNG, converted from a specific DICOM study series. Furthermore, the voxel spacings most likely have been re-sampled to be uniform for all images. When integrated with imaging networks and receiving DICOM instances from modalities and Picture Archiving and Communications System, PACS, an AI deploy application may have to deal with a whole DICOM study with multiple series, whose images' spacing may not be the same as expected by the trained model. To address these cases consistently and efficiently, MONAI Deploy Application SDK provides classes, called operators, to parse DICOM studies, select specific series with application defined rules, and convert the selected DICOM series into domain specific image format along with meta-data representing the pertinent DICOM attributes.

In the following sections, we will demonstrate how to create an MONAI Deploy application package using the MONAI Deploy App SDK.

:::{note}
For local testing, if there is a lack of DICOM Part 10 files, one can use open source programs, e.g. 3D Slicer, to convert NIfTI to DICOM files.

It is important that the DICOM files are present in the folder `dcm`, and the trained Torch Script model is at `model/spleen_model.ts`
:::

## Creating Operators and connecting them in Application class

We will implement an application that consists of five Operators:

- **DICOMDataLoaderOperator**:
    - **Input(dicom_files)**: a folder path ([`DataPath`](/modules/_autosummary/monai.deploy.core.domain.DataPath))
    - **Output(dicom_study_list)**: a list of DICOM studies in memory (List[[`DICOMStudy`](/modules/_autosummary/monai.deploy.core.domain.DICOMStudy)])
- **DICOMSeriesSelectorOperator**:
    - **Input(dicom_study_list)**: a list of DICOM studies in memory (List[[`DICOMStudy`](/modules/_autosummary/monai.deploy.core.domain.DICOMStudy)])
    - **Input(selection_rules)**: a selection rule (Dict)
    - **Output(dicom_series)**: a DICOM series object in memory ([`DICOMSeries`](/modules/_autosummary/monai.deploy.core.domain.DICOMSeries))
- **DICOMSeriesToVolumeOperator**:
    - **Input(dicom_series)**: a DICOM series object in memory ([`DICOMSeries`](/modules/_autosummary/monai.deploy.core.domain.DICOMSeries))
    - **Output(image)**: an image object in memory ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
- **SpleenSegOperator**:
    - **Input(image)**: an image object in memory ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
    - **Output(seg_image)**: an image object in memory ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
- **DICOMSegmentationWriterOperator**:
    - **Input(seg_image)**: a segmentation image object in memory ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
    - **Input(dicom_series)**: a DICOM series object in memory ([`DICOMSeries`](/modules/_autosummary/monai.deploy.core.domain.DICOMSeries))
    - **Output(dicom_seg_instance)**: a file path ([`DataPath`](/modules/_autosummary/monai.deploy.core.domain.DataPath))


:::{note}
The `DICOMSegmentationWriterOperator` needs both the segmentation image as well as the original DICOM series meta-data in order to use the patient demographics and the DICOM Study level attributes.
:::

The workflow of the application would look like this.

```{mermaid}
%%{init: {"theme": "base", "themeVariables": { "fontSize": "16px"}} }%%

classDiagram
    direction TB

    DICOMDataLoaderOperator --|> DICOMSeriesSelectorOperator : dicom_study_list...dicom_study_list
    DICOMSeriesSelectorOperator --|> DICOMSeriesToVolumeOperator : dicom_series...dicom_series
    DICOMSeriesToVolumeOperator --|> SpleenSegOperator : image...image
    DICOMSeriesSelectorOperator --|> DICOMSegmentationWriterOperator : dicom_series...dicom_series
    SpleenSegOperator --|> DICOMSegmentationWriterOperator : seg_image...seg_image


    class DICOMDataLoaderOperator {
        <in>dicom_files : DISK
        dicom_study_list(out) IN_MEMORY
    }
    class DICOMSeriesSelectorOperator {
        <in>dicom_study_list : IN_MEMORY
        <in>selection_rules : IN_MEMORY
        dicom_series(out) IN_MEMORY
    }
    class DICOMSeriesToVolumeOperator {
        <in>dicom_series : IN_MEMORY
        image(out) IN_MEMORY
    }
    class SpleenSegOperator {
        <in>image : IN_MEMORY
        seg_image(out) IN_MEMORY
    }
    class DICOMSegmentationWriterOperator {
        <in>seg_image : IN_MEMORY
        <in>dicom_series : IN_MEMORY
        dicom_seg_instance(out) DISK
    }
```

### Setup environment


In [85]:
# Install MONAI and other necessary image processing packages for the application
!python -c "import monai" || pip install -q "monai"
!python -c "import torch" || pip install -q "torch>=1.5"
!python -c "import numpy" || pip install -q "numpy>=1.17"
!python -c "import nibabel" || pip install -q "nibabel>=3.2.1"
!python -c "import pydicom" || pip install -q "pydicom>=1.4.2"
!python -c "import SimpleITK" || pip install -q "SimpleITK>=2.0.0"
!python -c "import typeguard" || pip install -q "typeguard>=2.12.1"

# Install MONAI Deploy App SDK package
!python -c "import monai.deploy" || pip install -q "monai-deploy-app-sdk"

### Setup imports

Let's import necessary classes/decorators to define Application and Operator.

In [86]:
import logging
from os import path

from numpy import uint8

import monai.deploy.core as md
from monai.deploy.core import ExecutionContext, Image, InputContext, IOType, Operator, OutputContext
from monai.deploy.operators.monai_seg_inference_operator import InMemImageReader, MonaiSegInferenceOperator
from monai.transforms import (
    Activationsd,
    AsDiscreted,
    Compose,
    CropForegroundd,
    EnsureChannelFirstd,
    Invertd,
    LoadImaged,
    SaveImaged,
    ScaleIntensityRanged,
    Spacingd,
    ToTensord,
)

from monai.deploy.core import Application, resource
from monai.deploy.operators.dicom_data_loader_operator import DICOMDataLoaderOperator
from monai.deploy.operators.dicom_seg_writer_operator import DICOMSegmentationWriterOperator
from monai.deploy.operators.dicom_series_selector_operator import DICOMSeriesSelectorOperator
from monai.deploy.operators.dicom_series_to_volume_operator import DICOMSeriesToVolumeOperator

### Creating Model Specific Inference Operator classes

Each Operator class inherits [Operator](/modules/_autosummary/monai.deploy.core.Operator) class and input/output properties are specified by using [@input](/modules/_autosummary/monai.deploy.core.input)/[@output](/modules/_autosummary/monai.deploy.core.output) decorators.

Business logic would be implemented in the <a href="/modules/_autosummary/monai.deploy.core.Operator.html#monai.deploy.core.Operator.compute">compute()</a> method.

The App SDK provide a `MonaiSegInferenceOperator` class to perform segmentation prediction with a Torch Script model. For consistency, this class uses MONAI dictionary based transforms, as `Compose` object, for pre and post transforms. The model specific inference operator will then only need to create the pre and post transform `Compose` based on what have been used in the model training and validation. Note that for deploy application, `ignite` is not needed nor supported.

#### SpleenSegOperator

The `SpleenSegOperator` gets as input an in-memory [Image](/modules/_autosummary/monai.deploy.core.domain.Image) object that has been converted from a DICOM CT series by the preceding `DICOMSeriesToVolumeOperator`, and as output in-memory segmentation [Image](/modules/_autosummary/monai.deploy.core.domain.Image) object.

The `pre_process` function creates the pre-tranforms `Compose` object. For `LoadImage`, a specialized `InMemImageReader`, derived from MONAI `ImageReader`, is used to convert the in-memory pixel data and return the `numpy` array as well as the meta-data. Also, the DICOM input pixel spacings are often not the same as expected by the model, so the `Spacingd` transform must be used to re-sample the image with the expected spacing.

The `post_process` function creates the post-transform `Compose` object. The `SaveImageD` transform class is used to save the segmentation mask as NIfTI image file, which is optional as the in-memory mask image will be passed down to the DICOM Segmentation writer for creating an DICOM Segmentation instance. The `Invertd` must also be used to revert the segmentation image's orientation and spacing to be the same as the input.

When the `MonaiSegInferenceOperator` object is created, the `ROI` size is specified, as well as the transform `Compose` objects. Furthermore, the dataset image key names are set accordingly.

Loading of the model and performing the prediction are encapsulated in the `MonaiSegInferenceOperator` and other SDK classes. Once inference is completed, the segmentation [Image](/modules/_autosummary/monai.deploy.core.domain.Image) object is created and set to the output (<a href="/modules/_autosummary/monai.deploy.core.OutputContext.html#monai.deploy.core.OutputContext.set">op_output.set(value, label)</a>), by the `MonaiSegInferenceOperator`.

In [87]:
@md.input("image", Image, IOType.IN_MEMORY)
@md.output("seg_image", Image, IOType.IN_MEMORY)
@md.env(pip_packages=["monai==0.6.0", "torch>=1.5", "numpy>=1.17", "nibabel"])
class SpleenSegOperator(Operator):
    """Performs Spleen segmentation with a 3D image converted from a DICOM CT series.
    """

    def __init__(self):

        self.logger = logging.getLogger("{}.{}".format(__name__, type(self).__name__))
        super().__init__()
        self._input_dataset_key = "image"
        self._pred_dataset_key = "pred"

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):

        input_image = op_input.get("image")
        if not input_image:
            raise ValueError("Input image is not found.")

        output_path = context.output.get().path

        # This operator gets an in-memory Image object, so a specialized ImageReader is needed.
        _reader = InMemImageReader(input_image)
        pre_transforms = self.pre_process(_reader)
        post_transforms = self.post_process(pre_transforms, path.join(output_path, "prediction_output"))

        # Delegates inference and saving output to the built-in operator.
        infer_operator = MonaiSegInferenceOperator(
            (
                160,
                160,
                160,
            ),
            pre_transforms,
            post_transforms,
        )

        # Setting the keys used in the dictironary based transforms may change.
        infer_operator.input_dataset_key = self._input_dataset_key
        infer_operator.pred_dataset_key = self._pred_dataset_key

        # Now let the built-in operator handles the work with the I/O spec and execution context.
        infer_operator.compute(op_input, op_output, context)

    def pre_process(self, img_reader) -> Compose:
        """Composes transforms for preprocessing input before predicting on a model."""

        my_key = self._input_dataset_key
        return Compose(
            [
                LoadImaged(keys=my_key, reader=img_reader),
                EnsureChannelFirstd(keys=my_key),
                Spacingd(keys=my_key, pixdim=[1.0, 1.0, 1.0], mode=["blinear"], align_corners=True),
                ScaleIntensityRanged(keys=my_key, a_min=-57, a_max=164, b_min=0.0, b_max=1.0, clip=True),
                CropForegroundd(keys=my_key, source_key=my_key),
                ToTensord(keys=my_key),
            ]
        )

    def post_process(self, pre_transforms: Compose, out_dir: str = "./prediction_output") -> Compose:
        """Composes transforms for postprocessing the prediction results."""

        pred_key = self._pred_dataset_key
        return Compose(
            [
                Activationsd(keys=pred_key, softmax=True),
                AsDiscreted(keys=pred_key, argmax=True),
                Invertd(
                    keys=pred_key, transform=pre_transforms, orig_keys=self._input_dataset_key, nearest_interp=True
                ),
                SaveImaged(keys=pred_key, output_dir=out_dir, output_postfix="seg", output_dtype=uint8, resample=False),
            ]
        )


### Creating Application class

Our application class would look like below.

It defines `App` class, inheriting [Application](/modules/_autosummary/monai.deploy.core.Application) class.

The requirements (resource and package dependency) for the App can be specified by using [@resource](/modules/_autosummary/monai.deploy.core.resource) and [@env](/modules/_autosummary/monai.deploy.core.env) decorators.

The base class method, `compose`, is overridden. Objects required for DICOM parsing, series selection (selecting the first series for the current release), pixel data conversion to volume image, and segmentation instance creation are created, so is the model specific `SpleenSegOperator`. The execution pipeline, as a Directed Acyclic Graph, is created by connecting these objects through <a href="/modules/_autosummary/monai.deploy.core.Application.html#monai.deploy.core.Application.add_flow">self.add_flow()</a>.

In [88]:
@resource(cpu=1, gpu=1, memory="7Gi")
class AISpleenSegApp(Application):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def compose(self):

        study_loader_op = DICOMDataLoaderOperator()
        series_selector_op = DICOMSeriesSelectorOperator()
        series_to_vol_op = DICOMSeriesToVolumeOperator()
        # Creates DICOM Seg writer with segment label name in a string list
        dicom_seg_writer = DICOMSegmentationWriterOperator(seg_labels=["Spleen"])

        # Creates the model specific segmentation operator
        spleen_seg_op = SpleenSegOperator()

        # Creates the DAG by linking the operators
        self.add_flow(study_loader_op, series_selector_op, {"dicom_study_list": "dicom_study_list"})
        self.add_flow(series_selector_op, series_to_vol_op, {"dicom_series": "dicom_series"})
        self.add_flow(series_to_vol_op, spleen_seg_op, {"image": "image"})

        self.add_flow(series_selector_op, dicom_seg_writer, {"dicom_series": "dicom_series"})
        self.add_flow(spleen_seg_op, dicom_seg_writer, {"seg_image": "seg_image"})


## Executing app locally

We can execute the app in the Jupyter notebook. Note that the DICOM files of the CT Abdomen series must be present in the `DICOM_folder` and the Torch Script model at `model_file`. Please use the actual path in your environment.

`app = AISpleenSegApp()`

`app.run(input="dcm", model="model/spleen_model.ts", output="output")`

Once the application is verified inside Jupyter notebook, we can write the above Python code into Python files in an application folder.

The application folder structure would look like below:

```bash
my_app
├── __main__.py
├── app.py
└── spleen_seg_operator.py
```

:::{note}
We can create a single application Python file (such as `spleen_app.py`) that includes the content of the files, instead of creating multiple files.
You will see such example in <a href="/notebooks/tutorials/02_mednist_app.html#executing-app-locally">MedNist Classifier Tutorial</a>.
:::

In [89]:
# Create an application folder
!mkdir -p my_app

### spleen_seg_operator.py

In [90]:
%%writefile my_app/spleen_seg_operator.py
import logging
from os import path

from numpy import uint8

import monai.deploy.core as md
from monai.deploy.core import ExecutionContext, Image, InputContext, IOType, Operator, OutputContext
from monai.deploy.operators.monai_seg_inference_operator import InMemImageReader, MonaiSegInferenceOperator
from monai.transforms import (
    Activationsd,
    AsDiscreted,
    Compose,
    CropForegroundd,
    EnsureChannelFirstd,
    Invertd,
    LoadImaged,
    SaveImaged,
    ScaleIntensityRanged,
    Spacingd,
    ToTensord,
)


@md.input("image", Image, IOType.IN_MEMORY)
@md.output("seg_image", Image, IOType.IN_MEMORY)
@md.env(pip_packages=["monai==0.6.0", "torch>=1.5", "numpy>=1.17", "nibabel", "typeguard"])
class SpleenSegOperator(Operator):
    """Performs Spleen segmentation with a 3D image converted from a DICOM CT series.
    """

    def __init__(self):

        self.logger = logging.getLogger("{}.{}".format(__name__, type(self).__name__))
        super().__init__()
        self._input_dataset_key = "image"
        self._pred_dataset_key = "pred"

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):

        input_image = op_input.get("image")
        if not input_image:
            raise ValueError("Input image is not found.")

        output_path = context.output.get().path

        # This operator gets an in-memory Image object, so a specialized ImageReader is needed.
        _reader = InMemImageReader(input_image)
        pre_transforms = self.pre_process(_reader)
        post_transforms = self.post_process(pre_transforms, path.join(output_path, "prediction_output"))

        # Delegates inference and saving output to the built-in operator.
        infer_operator = MonaiSegInferenceOperator(
            (
                160,
                160,
                160,
            ),
            pre_transforms,
            post_transforms,
        )

        # Setting the keys used in the dictironary based transforms may change.
        infer_operator.input_dataset_key = self._input_dataset_key
        infer_operator.pred_dataset_key = self._pred_dataset_key

        # Now let the built-in operator handles the work with the I/O spec and execution context.
        infer_operator.compute(op_input, op_output, context)

    def pre_process(self, img_reader) -> Compose:
        """Composes transforms for preprocessing input before predicting on a model."""

        my_key = self._input_dataset_key
        return Compose(
            [
                LoadImaged(keys=my_key, reader=img_reader),
                EnsureChannelFirstd(keys=my_key),
                Spacingd(keys=my_key, pixdim=[1.0, 1.0, 1.0], mode=["blinear"], align_corners=True),
                ScaleIntensityRanged(keys=my_key, a_min=-57, a_max=164, b_min=0.0, b_max=1.0, clip=True),
                CropForegroundd(keys=my_key, source_key=my_key),
                ToTensord(keys=my_key),
            ]
        )

    def post_process(self, pre_transforms: Compose, out_dir: str = "./prediction_output") -> Compose:
        """Composes transforms for postprocessing the prediction results."""

        pred_key = self._pred_dataset_key
        return Compose(
            [
                Activationsd(keys=pred_key, softmax=True),
                AsDiscreted(keys=pred_key, argmax=True),
                Invertd(
                    keys=pred_key, transform=pre_transforms, orig_keys=self._input_dataset_key, nearest_interp=True
                ),
                SaveImaged(keys=pred_key, output_dir=out_dir, output_postfix="seg", output_dtype=uint8, resample=False),
            ]
        )


Overwriting my_app/spleen_seg_operator.py


### app.py

In [91]:
%%writefile my_app/app.py
import logging

from spleen_seg_operator import SpleenSegOperator

from monai.deploy.core import Application, resource
from monai.deploy.operators.dicom_data_loader_operator import DICOMDataLoaderOperator
from monai.deploy.operators.dicom_seg_writer_operator import DICOMSegmentationWriterOperator
from monai.deploy.operators.dicom_series_selector_operator import DICOMSeriesSelectorOperator
from monai.deploy.operators.dicom_series_to_volume_operator import DICOMSeriesToVolumeOperator

@resource(cpu=1, gpu=1, memory="7Gi")
class AISpleenSegApp(Application):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def compose(self):

        study_loader_op = DICOMDataLoaderOperator()
        series_selector_op = DICOMSeriesSelectorOperator()
        series_to_vol_op = DICOMSeriesToVolumeOperator()
        # Creates DICOM Seg writer with segment label name in a string list
        dicom_seg_writer = DICOMSegmentationWriterOperator(seg_labels=["Spleen"])
        # Creates the model specific segmentation operator
        spleen_seg_op = SpleenSegOperator()

        # Creates the DAG by link the operators
        self.add_flow(study_loader_op, series_selector_op, {"dicom_study_list": "dicom_study_list"})
        self.add_flow(series_selector_op, series_to_vol_op, {"dicom_series": "dicom_series"})
        self.add_flow(series_to_vol_op, spleen_seg_op, {"image": "image"})
        self.add_flow(series_selector_op, dicom_seg_writer, {"dicom_series": "dicom_series"})
        self.add_flow(spleen_seg_op, dicom_seg_writer, {"seg_image": "seg_image"})

if __name__ == "__main__":
    # Creates the app and test it standalone. When running is this mode, please note the following:
    #     -m <model file>, for model file path
    #     -i <DICOM folder>, for input DICOM CT series folder
    #     -o <output folder>, for the output folder, default $PWD/output
    # e.g.
    #     python3 app.py -i input -m model/model.ts
    #
    AISpleenSegApp(do_run=True)

Overwriting my_app/app.py


```python
if __name__ == "__main__":
    AISpleenSegApp(do_run=True)
```

Above lines are needed to execute the application code by using `python` interpreter.

### \_\_main\_\_.py

\_\_main\_\_.py is needed for <a href="/developing_with_sdk/packaging_app#required-arguments">MONAI Application Packager</a> to detect main application code (`app.py`) when the application is executed with the application folder path (e.g., `python simple_imaging_app`).

In [92]:
%%writefile my_app/__main__.py
from app import AISpleenSegApp

if __name__ == "__main__":
    AISpleenSegApp(do_run=True)

Overwriting my_app/__main__.py


In [93]:
!ls my_app

app.py	app.pyc  __main__.py  __pycache__  spleen_seg_operator.py


In this time, let's execute the app in the command line.

In [94]:
!python my_app -i dcm -m model/spleen_model.ts -o output

Going to initiate execution of operator DICOMDataLoaderOperator
Executing operator DICOMDataLoaderOperator (Process ID: 17033, Operator ID: ff94ae9f-48d7-4a28-9282-414c53d1a609)
Done performing execution of operator DICOMDataLoaderOperator

Going to initiate execution of operator DICOMSeriesSelectorOperator
Executing operator DICOMSeriesSelectorOperator (Process ID: 17033, Operator ID: 449536b0-56fb-4761-8bad-04297eb15253)
Done performing execution of operator DICOMSeriesSelectorOperator

Going to initiate execution of operator DICOMSeriesToVolumeOperator
Executing operator DICOMSeriesToVolumeOperator (Process ID: 17033, Operator ID: e982a107-7946-407b-ad34-f5fd84168d66)
Done performing execution of operator DICOMSeriesToVolumeOperator

Going to initiate execution of operator SpleenSegOperator
Executing operator SpleenSegOperator (Process ID: 17033, Operator ID: 5bf44d57-8939-474f-8352-b54b7a4b7473)
file written: /home/mqin/src/monai-app-sdk/notebooks/tutorials/output/prediction_output

Above command is same with the following command line:

In [95]:
!monai-deploy exec my_app -i dcm -m model/spleen_model.ts -o output

Going to initiate execution of operator DICOMDataLoaderOperator
Executing operator DICOMDataLoaderOperator (Process ID: 17141, Operator ID: a268fd76-5bbd-4d85-9d9f-89776403b7f1)
Done performing execution of operator DICOMDataLoaderOperator

Going to initiate execution of operator DICOMSeriesSelectorOperator
Executing operator DICOMSeriesSelectorOperator (Process ID: 17141, Operator ID: 253ff9c8-18a7-451f-a73f-80fd43df94e2)
Done performing execution of operator DICOMSeriesSelectorOperator

Going to initiate execution of operator DICOMSeriesToVolumeOperator
Executing operator DICOMSeriesToVolumeOperator (Process ID: 17141, Operator ID: e84f7fd6-c717-46a8-bcf7-4e41acb99cc5)
Done performing execution of operator DICOMSeriesToVolumeOperator

Going to initiate execution of operator SpleenSegOperator
Executing operator SpleenSegOperator (Process ID: 17141, Operator ID: 95474b01-fbf3-4721-9bb0-a232f44916c3)
file written: /home/mqin/src/monai-app-sdk/notebooks/tutorials/output/prediction_output

In [96]:
!ls output

dicom_seg-DICOMSEG.dcm	prediction_output


## Packaging app

Let's package the app with [MONAI Application Packager](/developing_with_sdk/packaging_app).

In [97]:
!monai-deploy package my_app --tag my_app:latest -m model/spleen_model.ts

Building MONAI Application Package... Done
[2021-09-23 00:01:47,765] [INFO] (app_packager) - Successfully built my_app:latest


:::{note}
Building a MONAI Application Package (Docker image) can take time. Use `-l DEBUG` option if you want to see the progress.
:::

We can see that the Docker image is created.

In [98]:
!docker image ls | grep my_app

my_app                                                            latest                     8977e30d8f33   15 minutes ago   15.6GB
my_app                                                            0.0.1                      8d82ae089d98   2 days ago       15.6GB


## Executing packaged app locally

The packaged app can be run locally through [MONAI Application Runner](/developing_with_sdk/executing_packaged_app_locally).

In [99]:
# Copy DICOM files are in 'dcm' folder

# Launch the app
!monai-deploy run my_app:latest dcm output

Checking dependencies...
--> Verifying if "docker" is installed...

--> Verifying if "my_app:latest" is available...

Checking for MAP "my_app:latest" locally
"my_app:latest" found.

Reading MONAI App Package manifest...
 > export '/var/run/monai/export/' detected
--> Verifying if "nvidia-docker" is installed...

Going to initiate execution of operator DICOMDataLoaderOperator
Executing operator DICOMDataLoaderOperator (Process ID: 1, Operator ID: 834c7259-377d-41f2-a293-d1cd57b6bf55)
Done performing execution of operator DICOMDataLoaderOperator

Going to initiate execution of operator DICOMSeriesSelectorOperator
Executing operator DICOMSeriesSelectorOperator (Process ID: 1, Operator ID: d6077359-8953-4373-95e1-4e67e425f456)
Done performing execution of operator DICOMSeriesSelectorOperator

Going to initiate execution of operator DICOMSeriesToVolumeOperator
Executing operator DICOMSeriesToVolumeOperator (Process ID: 1, Operator ID: 4486a387-3838-4e67-9b38-02aa1371d616)
Done performing e

In [100]:
!ls output

dicom_seg-DICOMSEG.dcm	prediction_output
